# *ml-1E-dask-distributed-model-runs.ipynb*

This notebook (1E) features the most successful way I've found using dask.

# Running models in parallel

This is a work-in-progress-notebook. Currently, it allows us to: specify a models bank for 160 Random Forest and Extra Trees models; assemble the datasets for different folds of year-wise cross-validation; run the models for 12 combinations of region, crop, compositing scheme, and in-season date; and record results (accuracies and uncertainties) in csv files.

Function definitions at the top of the notebook; then sequential model runs; then parallel model runs.

New in version 1D: parallel model runs using Dask.

New in version 1C: can draw from bank of specified models, and run through them in a for loop.

New in version 1B: can now specify `in_season`, which is None (meaning use whole year), or 230 (thru DOY 230 which is mid-August), or 160 (thru DOY 160 which is early June)

In [1]:
import numpy as np
import pandas as pd
import os

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler

In [2]:
def create_models_bank():
    models_bank = {}
    increment = 0
    for n_estimators in [200, 500]:
        for max_features in [0.05, 0.1, 0.2, 0.4, 1.0]:
            for min_samples_split in [2, 4]:
                for bootstrap in [False, True]:
                    for class_weight in [None, 'balanced']:

                        increment += 1

                        three_digit = str(increment).zfill(3)
                        models_bank[three_digit] = {
                            'n_estimators': n_estimators,
                            'max_features': max_features,
                            'bootstrap': bootstrap,
                            'min_samples_split': min_samples_split,
                            'class_weight': class_weight,
                            'n_jobs': -1
                        }

    return models_bank

In [3]:
def create_X_y_single_year(tile,year,scheme_name,crop_of_interest_id,
                          in_season=None):
    coiid = crop_of_interest_id
    
    refl = np.load(f'../data/composited_interpolated/Refl_{tile}_{year}_{scheme_name}.npy')

    # Determine nrf (number of reflectance features)
    if in_season not in [160, 230, None]:
        print('Please change in_season to 160, 230, or None.')
        return None
    if in_season is None:
        nrf = refl.shape[1]
    if in_season in [160, 230]:
        pdsize = int(scheme_name[:-3])
        nrf = ((in_season - 90) // pdsize) * 6 + 6
        
    crop = []
    for y in range(year-4,year+1):
        crop.append(np.load(f'../data/processed_crop/Crop_{tile}_{y}.npy')==coiid)

    X = np.column_stack([refl[:,:nrf]] + crop[:-1])
    y = crop[-1]
    
    return X, y

In [4]:
def create_X_y_multiyear(tile,
                      years,
                      scheme_name,
                      crop_of_interest_id,
                        in_season=None):
    X_list = []
    y_list = []
    
    for y in years:
        X, y = create_X_y_single_year(tile,y,scheme_name,
                                      crop_of_interest_id,
                                     in_season)
        X_list.append(X)
        y_list.append(y)
    
    X = np.concatenate(X_list)
    y = np.concatenate(y_list)
    
    return X, y

In [5]:
def create_X_y(tile,
              years,
              scheme_name,
              crop_of_interest_id,
              in_season=None
              ):
    if type(years)==int:
        return create_X_y_single_year(tile,
                                      years,
                                      scheme_name,
                                      crop_of_interest_id,
                                      in_season)
    
    return create_X_y_multiyear(tile,
                              years,
                              scheme_name,
                              crop_of_interest_id,
                              in_season)

In [6]:
def fit_predict_report(model_name,
                      model,
                      training_sample_size,
                      validation_sample_size,
                      tile,
                      years,
                      scheme_name,
                      crop_of_interest_id,
                      in_season
                      ):
    
    # produce csv_name
    exempt = ['years', 'model']
    param_value_strings = [f'{model_name}',
                      f'{training_sample_size}',
                      f'{validation_sample_size}',
                      f'{tile}',
                      f'{scheme_name}',
                      f'{crop_of_interest_id}',
                      f'{in_season}']
    csv_name = '_'.join(param_value_strings) +'.csv'

    # check whether previously run and, if so, end the effort
    if csv_name in os.listdir('../data/results/'):
        return 'If you see this, the specified model was previously run.'

    print(f'-- Process for {csv_name} --')
    
    # below is actually fitting and predicting and reporting
    
    conf = []

    for val_year in years:
        print('Starting a fold...')
        print('> Assembling the datasets')
        train_years = [yr for yr in range(2018,2023) if yr!=val_year]

        X_train0, y_train0 = create_X_y(tile,train_years,
                                      scheme_name,crop_of_interest_id,
                                       in_season)
        X_val0, y_val0 = create_X_y(tile,val_year,
                                  scheme_name,crop_of_interest_id,
                                  in_season)

        if training_sample_size is not None:
            X_train, X_trsurplus, y_train, y_trsurplus = train_test_split(X_train0,
                                                                     y_train0,
                                                                     train_size=training_sample_size,
                                                                     random_state=19)
        if training_sample_size is None:
            X_train, y_train = X_train0, y_train0

        if validation_sample_size is not None:
            X_val, X_vsurplus, y_val, y_vsurplus = train_test_split(X_val0,
                                                                     y_val0,
                                                                     train_size=validation_sample_size,
                                                                     random_state=19)
        if validation_sample_size is None:
            X_val, y_val = X_val0, y_val0
    
            
        print('> Fitting the model on the training set')
        model.fit(X_train, y_train)
        print('> Predicting on the validation set')
        pred = model.predict(X_val)

        print('> Recording performance metrics')
        act = y_val
        ActPred_00 = sum((act==0) & (pred==0))
        ActPred_01 = sum((act==0) & (pred==1))
        ActPred_10 = sum((act==1) & (pred==0))
        ActPred_11 = sum((act==1) & (pred==1))
        conf_1yr = [ActPred_00, ActPred_01, ActPred_10, ActPred_11]

        conf.append(conf_1yr)
        print('Finished a fold.')

    carr = np.array(conf)

    carr = np.row_stack([carr,np.full((2,4),-1)])

    # above we added the totals row
    # now we need to add the columns for precision and recall

    # create dataframe
    cdf = pd.DataFrame(data = carr,
                      index = [f'ValYear{yr}' for yr in years]+['Mean','StdE'],
                      columns = ['ActPred_00', 'ActPred_01', 
                                 'ActPred_10', 'ActPred_11']
                      )

    cdf['Precision'] = cdf.ActPred_11 / (cdf.ActPred_01 + cdf.ActPred_11)
    cdf['Recall'] = cdf.ActPred_11 / (cdf.ActPred_10 + cdf.ActPred_11)
    cdf['F1'] = 2*cdf.Precision*cdf.Recall / (cdf.Precision + cdf.Recall)
    for col in ['Precision','Recall','F1']:
        cdf.at['Mean',col] = np.mean(cdf.loc[:'ValYear2022',col])
        cdf.at['StdE',col] = np.std(cdf.loc[:'ValYear2022',col])
    
    
    param_strings = [f'# model_name: {model_name}',
                     f'# model: {model}',
                      f'# training_sample_size: {training_sample_size}',
                      f'# validation_sample_size: {validation_sample_size}',
                      f'# tile: {tile}',
                      f'# scheme_name: {scheme_name}',
                      f'# crop_of_interest_id: {crop_of_interest_id}',
                      f'# in_season: {in_season}']
    comment = '\n'.join(param_strings) + '\n' 
    with open(f'../data/results/{csv_name}', 'a') as f:
        f.write(comment)
        cdf.to_csv(f)
    
    print(f'Find results in ../data/results/{csv_name}')
    
    return f'Find results in ../data/results/{csv_name}'

## Dask version

Still troubleshooting memory issues on MGHPCC. Given enough memory resources this should run.

In [7]:
import dask
from dask.distributed import print
from dask.distributed import Client

In [8]:
client = Client(memory_limit='128GiB',
                threads_per_worker=1,
               )

distributed.diskutils - INFO - Found stale lock file and directory '/work/sds-lab/august/crops/notebooks/dask-worker-space/worker-0lae_gks', purging
distributed.diskutils - INFO - Found stale lock file and directory '/work/sds-lab/august/crops/notebooks/dask-worker-space/worker-_jkc0fpy', purging
distributed.diskutils - INFO - Found stale lock file and directory '/work/sds-lab/august/crops/notebooks/dask-worker-space/worker-c1of6rnf', purging
distributed.diskutils - INFO - Found stale lock file and directory '/work/sds-lab/august/crops/notebooks/dask-worker-space/worker-io_5eoo_', purging
distributed.diskutils - INFO - Found stale lock file and directory '/work/sds-lab/august/crops/notebooks/dask-worker-space/worker-iulsl_ol', purging
distributed.diskutils - INFO - Found stale lock file and directory '/work/sds-lab/august/crops/notebooks/dask-worker-space/worker-sd5pa2jz', purging
distributed.diskutils - INFO - Found stale lock file and directory '/work/sds-lab/august/crops/notebooks/d

In [9]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 56
Total threads: 56,Total memory: 7.00 TiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:44757,Workers: 56
Dashboard: http://127.0.0.1:8787/status,Total threads: 56
Started: Just now,Total memory: 7.00 TiB
Comm: tcp://127.0.0.1:36440,Total threads: 1
Dashboard: http://127.0.0.1:44509/status,Memory: 128.00 GiB
Nanny: tcp://127.0.0.1:39406,


In [ ]:
#client.close()

In [10]:
fit_predict_report = dask.delayed(fit_predict_report)

## Regular parallel version:

In [11]:
models_bank = create_models_bank()
training_sample_size = 0.001
validation_sample_size = 0.001

r = []
for tile_coiid in [('10SFH',75),('15TVG',1)]:
    for scheme_name in ['14day','5day']:
        for in_season in [160, 230, None]:
            for three_digit in models_bank.keys():

                ## Parameters dictionary p RANDOM FOREST
                p = {

                ## SPECIFY MODEL ##
                'model_name': 'RF' + three_digit,
                'model': RandomForestClassifier(**models_bank[three_digit]),
                'training_sample_size': training_sample_size,
                'validation_sample_size': validation_sample_size,

                ## SPECIFY TILE AND SCHEME ##
                'tile': tile_coiid[0],
                'years': [2018, 2019, 2020, 2021, 2022],
                'scheme_name': scheme_name,
                'crop_of_interest_id': tile_coiid[1], 
                'in_season': in_season
                }

                #fit_predict_report(**p) # run with the above parameters
                r.append(fit_predict_report(**p))
                
                ## Parameters dictionary p EXTRA TREES
                p = {

                ## SPECIFY MODEL ##
                'model_name': 'ET' + three_digit,
                'model': ExtraTreesClassifier(**models_bank[three_digit]),
                'training_sample_size': training_sample_size,
                'validation_sample_size': validation_sample_size,

                ## SPECIFY TILE AND SCHEME ##
                'tile': tile_coiid[0],
                'years': [2018, 2019, 2020, 2021, 2022],
                'scheme_name': scheme_name,
                'crop_of_interest_id': tile_coiid[1], 
                'in_season': in_season
                }

                r.append(fit_predict_report(**p))
                
dask.compute(*r)

-- Process for RF001_0.001_0.001_15TVG_14day_1_160.csv --
Starting a fold...
> Assembling the datasets
-- Process for ET074_0.001_0.001_15TVG_5day_1_160.csv --
Starting a fold...
> Assembling the datasets
-- Process for ET029_0.001_0.001_10SFH_5day_75_None.csv --
Starting a fold...
> Assembling the datasets
-- Process for ET058_0.001_0.001_15TVG_14day_1_230.csv --
Starting a fold...
> Assembling the datasets
-- Process for ET074_0.001_0.001_15TVG_5day_1_230.csv --
Starting a fold...
> Assembling the datasets
-- Process for RF035_0.001_0.001_10SFH_5day_75_160.csv --
Starting a fold...
> Assembling the datasets
-- Process for ET042_0.001_0.001_15TVG_5day_1_None.csv --
Starting a fold...
> Assembling the datasets
-- Process for RF015_0.001_0.001_10SFH_5day_75_230.csv --
Starting a fold...
> Assembling the datasets
-- Process for RF067_0.001_0.001_10SFH_14day_75_None.csv --
Starting a fold...
> Assembling the datasets
-- Process for RF011_0.001_0.001_10SFH_5day_75_230.csv --
Starting a fol

distributed.worker - WARNING - Compute Failed
Function:  execute_task
args:      ((<function apply at 0x2b33e84d9820>, <function fit_predict_report at 0x2b33fb08e0d0>, [], (<class 'dict'>, [['model_name', 'RF001'], ['model', RandomForestClassifier(bootstrap=False, max_features=0.05, n_estimators=200,
                       n_jobs=-1)], ['training_sample_size', 0.001], ['validation_sample_size', 0.001], ['tile', '15TVG'], ['years', [2018, 2019, 2020, 2021, 2022]], ['scheme_name', '14day'], ['crop_of_interest_id', 1], ['in_season', 160]])))
kwargs:    {}
Exception: "MemoryError((13395600, 40), dtype('int16'))"



MemoryError: Unable to allocate 1022. MiB for an array with shape (13395600, 40) and data type int16

distributed.nanny - WARNING - Restarting worker


-- Process for RF067_0.001_0.001_10SFH_14day_75_None.csv --
Starting a fold...
> Assembling the datasets


...

> Fitting the model on the training set
> Fitting the model on the training set
> Predicting on the validation set
> Predicting on the validation set


/shared/centos7/anaconda3/2022.05/lib/python3.9/site-packages/sklearn/ensemble/_base.py:194: UserWarning: Loky-backed parallel loops cannot be called in a multiprocessing, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)
/shared/centos7/anaconda3/2022.05/lib/python3.9/site-packages/sklearn/ensemble/_base.py:194: UserWarning: Loky-backed parallel loops cannot be called in a multiprocessing, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)


> Recording performance metrics
Finished a fold.
Starting a fold...
> Assembling the datasets
> Recording performance metrics
Finished a fold.
Starting a fold...
> Assembling the datasets
> Fitting the model on the training set


/shared/centos7/anaconda3/2022.05/lib/python3.9/site-packages/sklearn/ensemble/_base.py:194: UserWarning: Loky-backed parallel loops cannot be called in a multiprocessing, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)


> Predicting on the validation set
> Recording performance metrics
Finished a fold.
Starting a fold...
> Assembling the datasets
